<a href="https://colab.research.google.com/github/rodjfg/los-fabulosos-pixelotls/blob/master/Example_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
!pip install --quiet git+https://github.com/rodjfg/los-fabulosos-pixelotls

In [3]:
from los_fabulosos_pixelotls.tools import load_raw_data


ModuleNotFoundError: ignored